In [1]:
import sys
import os

sys.path.insert(0, os.path.abspath('../lib'))

In [ ]:

"""
2q-RB with quasi-static noise timescale of single measurement. (Same noise fluctuation for each Clifford sequence.)
"""

import random
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import multiprocessing as mp
from twoqrb import *

# stochastic noise deviation
std_uu = 20000
std_ud = 20000
std_du = 20000
std_dd = 0

# std_uu = 0
# std_ud = 0
# std_du = 0
# std_dd = 0

def RB_single_seq(L, repetition=125, ini_error=None, rd_error=None):
    initial = error_initial_state(ini_error[0], ini_error[1], ini_error[2])
    num_uu = 0
    cliff_seq = random.choices(Cliff_decompose, k=L)  # Cliff_decompose 隨便可重複取L個
    for j in range(repetition):
        phase = [0, 0, 0, 0]
        np.random.seed()
        # energy fluctuation noise is quasi-static with timescale of single measurement (single sequence).
        noise = [np.random.normal(0.0, std_uu), np.random.normal(0.0, std_ud),
                 np.random.normal(0.0, std_du), np.random.normal(0.0, std_dd)]
        g1 = get_seq(cliff_seq, phase, noise=noise, noise_type=QUASI_STATIC, crosstalk=False)
        g2 = get_seq_inverse(cliff_seq, phase, noise=noise, noise_type=QUASI_STATIC, crosstalk=False)
        seq_k = g2 @ g1  # k_th

        final_state = seq_k @ initial @ seq_k.conj().T
        proj_measure = error_initial_state(rd_error[0], rd_error[1], rd_error[2])
        final_prob = abs(np.trace(proj_measure @ final_state))
        # print(final_prob)
        # uu_count = [1, 0]
        # a = random.choices(uu_count, weights=[final_prob, 1-final_prob], k=1)
        # num_uu = num_uu + a[0]
        num_uu += final_prob
    return num_uu / repetition


# define the gate length series chosen to run RB protocol
l1 = np.arange(1, 20, 1)
l2 = np.arange(20, 40, 2)
l3 = np.arange(40, 80, 10)
l4 = np.arange(80, 240, 20)
l5 = np.arange(240, 300, 40)
l6 = np.arange(300, 550, 50)
x = np.hstack((l1, l2, l3, l4, l5, l6))
# x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

y = []
yerr = []

# N = 2  # samples for each Cliff_decompose point (算標準差即是使用每個data point對應的N個數據)
K = 51  # choices of S sequence 相同長度 重複取k次不同seq(等同K_L參數)
s_re = 125  # repeated times for each sequence
initial_error = [0, 0, 0]   # [e_ud, e_du, e_dd]
readout_error = [0, 0, 0]

def RB_loop(L):
    return RB_single_seq(L, repetition=s_re, ini_error=initial_error, rd_error=readout_error)


if __name__ == '__main__':
    for m in x:
        a = [m]*K
        pool = mp.Pool()
        res = pool.map(RB_loop, a)  # RB rep
        y.append(np.mean(res))
        yerr.append(np.std(res))
        print("L = ", m, " is done.")
        print("F = ", np.mean(res), "; std dev = ", np.std(res))
        # print(np.std(res))
    pool.close()
    pool.join()

    # saving the results
    f5 = open('2q_RB_simu_L.pkl', 'wb')
    pickle.dump(x, f5)
    f5.close()

    f6 = open('2q_RB_simu_y.pkl', 'wb')
    pickle.dump(y, f6)
    f6.close()

    f7 = open('2q_RB_simu_yerr.pkl', 'wb')
    pickle.dump(yerr, f7)
    f7.close()

    # Fitting function
    def func(x, A, B, r):
        return A * (1 - 4 / 3 * r) ** x + B

    # def func(x, A):
    #     return  A * (1 - 4/3*0.053)**x + 0.25

    popt, pcov = curve_fit(func, x, y, p0=[1, 0, 0], bounds=(0, 1), maxfev=5000)
    # p0 is the guess of the parameters.
    # Guess B ~ 0 (ideally be 0.25) and r ~ 0 (no noise model now so r should be ultra low)
    print("F_Ciff = 1 - r = ", 1 - popt[2])
    print("A = ", popt[0])
    print("B = ", popt[1])

    plt.errorbar(x, y, yerr=yerr, fmt='o', markersize=4, capsize=3)
    plt.plot(x, func(x, *popt), 'r-')
    # plt.plot(x, func(x, 0.75, 0.25, 0.053), 'b-')
    plt.ylim(top=1.0)
    plt.xlabel("Number of Cliffords (L)")
    plt.ylabel("Proj. State Prob.")
    plt.title("Two-qubit RB Fitting (No Crosstalk Low Frequency")
    plt.show()


L =  1  is done.
F =  0.9692373476704398 ; std dev =  0.03372380467311611
L =  2  is done.
F =  0.9498820040032947 ; std dev =  0.028326554528254423
L =  3  is done.
F =  0.9413005228677801 ; std dev =  0.028595367756928263
L =  4  is done.
F =  0.9202177574310845 ; std dev =  0.04277901371316348
L =  5  is done.
F =  0.9133948070915793 ; std dev =  0.041473711292777805
L =  6  is done.
F =  0.8992636962102872 ; std dev =  0.04756987748328392
L =  7  is done.
F =  0.8904345505904449 ; std dev =  0.04550016215510837
L =  8  is done.
F =  0.8641029549942674 ; std dev =  0.048943741434639775
L =  9  is done.
F =  0.8616843433970522 ; std dev =  0.05107335141696471
L =  10  is done.
F =  0.8341896680207922 ; std dev =  0.053927645009317175
L =  11  is done.
F =  0.8349711399336192 ; std dev =  0.05814007701966858
L =  12  is done.
F =  0.8220398042354512 ; std dev =  0.05986943150183397
L =  13  is done.
F =  0.8097191305794271 ; std dev =  0.06907044419872417
L =  14  is done.
F =  0.8047